# SSG닷컴 이마트몰 카테고리별 베스트 상품 크롤링

# 사용 패키지

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import time
import datetime

from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

# 사전작업
- 1. 가격정보의 중복되는 데이터를 제거 및 불필요한 특수문자 제거를 위한 함수
- 2. 가격정보에서 쉼표 제거하는 함수
 - 1번 함수의 예시 
  - '할인적용가\n3,840\n원\n(￦3,840)'
  - 필요한 가격정보인 6~14부분만 슬라이싱한다. 
  - 할인정보의 따로 작업을 하지 않아도 되지만, 할인 가격정보의 인덱스가 필요할 경우를 대비해 함수에서는 따로 구분을 해준다.

- 3. 한 가지 제품 정보를 두 줄로 표현한 경우, 브랜드를 한줄로 표기하고 해당 인덱스를 제거해준다.

In [11]:
# 할인가격 중복 제거
# em.ssg_price 는 할인가격이 리스트에 별도로 기록되기 때문에, 정제를 하기 매우 어렵다
# div.cunit_price.notranslate : 여기에는 기존가격, 할인가격이 한줄로 표시되어 가공이 더 수월하다
def remove_number(x):
    for i in range(len(x)):
        # if문이 없어도 되지만, 할인정보가 있는 행을 구분하기 위해 넣어줬다.
        if '에누리' in x[i]:
            x[i] = x[i][0:25]
        x[i] = x[i][6:14].replace('\n','').replace('원','').replace('(','').replace('\\','').replace('￦','').replace('~','')
    return x

# 가격에 쉼표 제거
def remove_str(x):
    return x.replace(',','')

# product 가져올때 브랜드이름, 상품명 따로 표기된 리스트가 있다.
# [브랜드 이름 + 제품명] 의 형태로 변경하고, 브랜드 이름 인덱스를 제거한다.
# 추가되는 상품이 있을시, name_list에 추가만하면 된다.
def replace_name(x):
    name_list = ['오뚜기','농심','팔도','PEACOCK','풀무원','No Brand','삼양','맥심','동서식품(주)','서울우유','매일','코카콜라',
                 '크라운','오리온','오리온제과','해태','롯데','E-MART FRESH','빙그레','하겐다즈','나뚜루','하림','맛있닭','테미즈',
                 '교촌1991','오븐에빠진닭','에그파파','허닭','올품','매일유업','서울우유','남양유업',
                 'LG생활건강','다우니','헨켈','참그린','크리넥스','코디','이마트24','SHYRILLA','유한킴벌리','모나리자','깨끗한나라',
                 '수려한','더랩바이블랑두', '스킨푸드', '니베아','피지오겔','SCINIC','더바디샵','한스킨','온더바디','스웨덴 에그팩',
                 '바이오더마','셀리맥스','버츠비','안나홀츠','설화수','홀리카홀리카','아이오페','바디판타지','센카','Dr.G','콜마운틴',
                 '에뛰드하우스','이지바이','트렉스타','도이터','카멜백','사츠','파라고나(paragona)','아크테릭스','레키',
                 '디스커버리(discovery)','트렉스타레저타임','파쉬','SUPERFEET','[JAJU/자주]신세계인터내셔날','라스포르티바[LASPORTIVA]',
                 '캠프라인','넥스토치','제이린드버그','레드페이스','오스프리','스탠리','월드컵(WORLDCUP)','데카트론','마운틴스미스',
                 '우리가스토리','콜핑','이드나인(ID9)','K2(케이투)','제이큐','반스플라이','BFL','마운티아[MOUNTIA]','휴몬트',
                 '헬리콘텍스','핫한날','선데이에프터눈즈','피크나인','언더아머(UNDER ARMOUR)','듀이셀','블리스텍스','네파[NEPA]',
                 '블랙야크','닉왁스(NIKWAX)','아이비','씨투써밋','부리나','마운틴가이드','BUCK703','팩세이프(pacsafe)',
                 '로우알파인','엘캡','바주카','헬리콘텍스[Helikon-Tex]','버팔로','트레스패스','마우나케아','3M','웨스트우드','아로마티카',
                 '머렐','노스페이스(NORTHFACE)','마몽드','성분에디터','카밀','수월한','메카닉스웨어','마운틴이큅먼트','에스테라',
                 '반고(VANGO)','감동','으뜸한돈','해태아이스크림','농협안심한돈','에스트라','C&C','핫타임','툴레(THULE)',
                 '프레시지','Adidas','단터프','TOPPEO','울파워','포센스','엑스피크','존슨&존슨','EVERLAST','4몬스터',
                 'nc picks','핸드맥스','지벤','일리커피(해외직구)','홈스웰','카르닉','스페이버','날진','차코','몽벨','알트라',
                 '에스트라','맥스페디션','틸리','자연맛남','노블진','티라노','SNP','일리윤','마녀공장','윌슨','펠로우','Gregory',
                 '네파(NEPA)','스타벅스','몬츄라(MONTURA)','리쥬란 힐러','마운틴벨리','작센','테슬라','기타브랜드','726기어',
                 '노스페이스[NORTHFACE]','파이커택티컬','피코크소싱','오다닭','미루나무','JK스포츠','레드페이스_이마트','엑스바이오닉',
                 '소울시즌스','마모트','네오텍티컬','K2세이프티','초록마을','피죤','유니레버','비욘드','유세린','스카르파(SCARPA)',
                 '미즈온','BIGTEN','아우토반','한율','아침의뜰','워크센스','레인보우샵','더마비','마이핫(MYHOT)','비스비바',
                 '다농이네','이니스프리','아벤느','롱샴(LONGCHAMP)','라로슈포제','컬럼비아(COLUMBIA)','롯데제과','네스카페 돌체구스토',
                 '비오텀','밀레(MILLET)','탑팀기어','미샤','MiiR','데이타임','사계절 스패츠','휴고프로쉬','힐레베르그',
                 '미스테리월','온종일 핫팩','JARDIN','세바메드','딥퍼랑스','코몽트','아트박스','배핀','더마토리','우체국쇼핑',
                 '미래통상','Summit','피쉬(해외직구)','프리메라','바나브','팜슨','눅스','험토','슬레진저','바닐라코','드래곤디퓨전',
                 '윈터솔','ETN','쿤타','에스이랜드','랑콤','마운틴밸리','K2_케이투','테크니카','제주푸드마씸','도드람한돈',
                 'KOVEA(코베아)','일리(illy)','폴에디트3','마스터베이스','아로요','위즈맥스','호카오네오네','일우','백마','나이키',
                 '카멜로','버먼트','헤드앤숄더','해피바스','질레트','팬틴','실크테라피','BYON','엘라스틴'
                ]
            
    for i in range(len(x)-1):
        for name in name_list:
            if x[i] == name:
                x[i+1] = (name + ' ' + x[i+1])

    for i in range(len(x)):
        for name in name_list:
            try:
                if x[i] == name:
                    x.remove(name)
            except : continue

# 크롤링
- 총 12개 품목에 대한 best-list 데이터 프레임 생성 후 하나로 통합
- ✰ 한번에 2개 이상 카테고리 작업시, 데이터 로드가 되지 않아서 각 카테고리 작업별로 '20초' 간격을 둔다.
- 페이지가 많은 '뷰티케어', '등산용품'의 경우 동일 카테고리간 데이터 로드가 되지 않을 때가 있어서, 5페이지마다 '5초'씩 간격을 둔다

In [12]:
# 인기품목별 BEST list & link
best_list=['라면','과자','커피','제철과일','아이스크림','돼지고기','닭고기','우유','세제','화장지','뷰티케어','등산용품']
best_list_link=['1000020108','1000020109','1000020110','1000020111','1000020112','1000020113','1000020114','1000020115','1000020116','1000020117','1000020118','1000020119']

# 크롤링 일시에 따른 파일명 설정을 위한 date 변수 설정
date=str(datetime.date.today()).replace('-','')[2:]

# 현재 카테고리별 best상품을 모두 담아줄 데이터프레임 생성
best_total_df=pd.DataFrame()

# 품목별로 순환
for link in best_list_link:

    # 해당 카테고리 넣어둘 df 생성
    total_df=pd.DataFrame()

    # 한 카테고리 내에서 실행하는 반복문, 페이지수 20으로 설정
    for num in range(15):
        if (num == 3) or (num == 6) or (num == 9) or (num == 12): # 3페이지마다 한번씩 더 쉬어준다.
            time.sleep(3)
        else:    
            # url지정
            url=f'http://emart.ssg.com/best/ajaxGetMoreItemList.ssg?cornrId=1000016472&cornrSetId={link}&dispCtgId=&pageSize=12&page={num+1}'
            # 여기서 link는 라면, 과자, 커피 등의 정보이고 num은 동일 품목에서 더보기를 누를 때 나오는 페이지 정보다

            # requests 정보 설정
            # 현재 PC의 referer과 User-Agent 기입
            headers = { 
                        'referer': 'http://emart.ssg.com/best/main.ssg?Egnb=best',
                        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.54 Safari/537.36'
                      }

            seed = np.random.randint(100)
            np.random.seed(seed)    
            a = np.random.randint(5)
            time.sleep(a)
            try:
                response=requests.get(url, headers=headers)
            except:
                print('접속 실패')
                break

            # html 전환
            data = BeautifulSoup(response.text, 'html.parser')

            # 페이지별 이름, 가격, 평점 리스트 생성
            product = [name.text for name in data.select('em.tx_ko')]
            replace_name(product)   # 윗줄에서 정의한 제품명을 함수를 사용해 중복 데이터를 날려준다
            
            price = [coin.text.strip() for coin in data.select('div.cunit_price.notranslate')]
            remove_number(price)   # 위에서 정의한 함수를 사용해서 가격정보를 다듬는다.(할인으로 인한 중복 가격표기도 제거)
            
            rate = [star.text.strip().strip().replace('별점 ','').replace('점','') for star in data.select('div.rate_bg')]
            # 별점을 시각화나 데이터 작업에 용이하게 숫자만 남겨둔다
            
            # 페이지 정보가 없으면 현재 검색중인 카테고리 루프문 종료
            if not product:
                break;

            else:
                # 현재 페이지 데이터프레임 생성(1개의 카테고리)
                df = pd.DataFrame(columns=['product','price','rate'], data=list(zip(product,price,rate)))

                # price, 별점 정보를 숫자형으로 전환
                df['price'] = df['price'].apply(remove_str).astype(int)
                df['rate'] = df['rate'].astype(float)

                # 현재 페이지 데이터프레임를 현재 카테고리 데이터프레임에 업데이트
                temp = df
                total_df = pd.concat([total_df, temp])
                total_df['카테고리'] = f'{best_list[best_list_link.index(link)]}'
                
    # 카테고리별 작업 완료
    print(f'{best_list[best_list_link.index(link)]} 인기품목 크롤링 완료')

    # 전체 데이터프레임에 완료된 카테고리 데이터 업데이트
    total_temp = total_df
    best_total_df = pd.concat([best_total_df, total_temp])
    best_total_df.index = range(len(best_total_df.index)) # 인덱스 초기화
    best_total_df = best_total_df[['카테고리','product','price','rate']]
    print(f'전체 데이터에 {best_list[best_list_link.index(link)]} 카테고리 병합 완료')
    
    if link != '1000020119':
        print('다음 카테고리 크롤링 진행\n')
        time.sleep(17)         # 페이지 막힘 방지를 위해 17초 뒤에 다음 카테고리 실행
    else:
        print('***** 크롤링 작업 완료 *****\n')

# 전체 카테고리 작업분 저장
best_total_df.to_csv(f'/Users/ppangppang/Documents/dev/TIL/test_ssg_data/{date}_Best_product.csv',index=False)
# 백업
best_total_df.to_csv(f'/Users/ppangppang/Desktop/ssac/data_crwaling/ssg_test/databackup/{date}_Best_product.csv',index=False)
print('***** 데이터 저장 완료 *****')

라면 인기품목 크롤링 완료
전체 데이터에 라면 카테고리 병합 완료
다음 카테고리 크롤링 진행

과자 인기품목 크롤링 완료
전체 데이터에 과자 카테고리 병합 완료
다음 카테고리 크롤링 진행

커피 인기품목 크롤링 완료
전체 데이터에 커피 카테고리 병합 완료
다음 카테고리 크롤링 진행

제철과일 인기품목 크롤링 완료
전체 데이터에 제철과일 카테고리 병합 완료
다음 카테고리 크롤링 진행

아이스크림 인기품목 크롤링 완료
전체 데이터에 아이스크림 카테고리 병합 완료
다음 카테고리 크롤링 진행

돼지고기 인기품목 크롤링 완료
전체 데이터에 돼지고기 카테고리 병합 완료
다음 카테고리 크롤링 진행

닭고기 인기품목 크롤링 완료
전체 데이터에 닭고기 카테고리 병합 완료
다음 카테고리 크롤링 진행

우유 인기품목 크롤링 완료
전체 데이터에 우유 카테고리 병합 완료
다음 카테고리 크롤링 진행

세제 인기품목 크롤링 완료
전체 데이터에 세제 카테고리 병합 완료
다음 카테고리 크롤링 진행

화장지 인기품목 크롤링 완료
전체 데이터에 화장지 카테고리 병합 완료
다음 카테고리 크롤링 진행

뷰티케어 인기품목 크롤링 완료
전체 데이터에 뷰티케어 카테고리 병합 완료
다음 카테고리 크롤링 진행

등산용품 인기품목 크롤링 완료
전체 데이터에 등산용품 카테고리 병합 완료
***** 크롤링 작업 완료 *****

***** 데이터 저장 완료 *****


In [13]:
# 데이터 확인
best_total_df

,카테고리,product,price,rate
0,라면,[농심] 올리브 짜파게티 (140gx5입),4280,4.95
1,라면,[농심] 신라면 120gx5입,3680,4.95
2,라면,[농심] 육개장 사발면 (86gx6입),4500,4.95
3,라면,오뚜기 진라면 매운맛 120g*5입,3100,4.96
4,라면,[농심] 안성탕면 5입 625g (125g5입),2980,4.94
...,...,...,...,...
673,등산용품,[마모트] 파워스트레치 장갑,19000,3.00
674,등산용품,BUCK703 K2 Safety 소프트넥워머 LMW-16952,8500,5.00
675,등산용품,K2세이프티 K2 라이크빈 LB2-F138 패딩 동계자켓,94743,4.83
676,등산용품,트렉스타 가볍고 튼튼한 미드컷 경등산화 / 파크,69000,5.00


In [26]:
# 저장한 데이터를 불러오기
# df = pd.read_csv(f'/Users/ppangppang/Documents/dev/TIL/test_ssg_data/{date}_Best_product.csv')
# df.tail(20)

In [ ]:
# # 화장품, 등산에 좀 특이사항이 많으니 이것들 부터 확인

# best_list=['돼지고기','뷰티케어','등산용품']
# best_list_link=['1000020113','1000020118','1000020119']

# # 현재 카테고리별 best상품을 모두 담아줄 데이터프레임 생성
# best_total_df=pd.DataFrame()

# # 품목별로 순환
# for link in best_list_link:

#     # 해당 카테고리 넣어둘 df 생성
#     total_df=pd.DataFrame()

#     # 한 카테고리 내에서 실행하는 반복문, 페이지수 20으로 설정
#     for num in range(12):
#         if (num == 3) or (num == 6) or (num == 9): # 5페이지마다 한번씩 더 쉬어준다.
#             time.sleep(4)
#         else:    
#             # url지정
#             url=f'http://emart.ssg.com/best/ajaxGetMoreItemList.ssg?cornrId=1000016472&cornrSetId={link}&dispCtgId=&pageSize=12&page={num+1}'
#             # 여기서 link는 라면, 과자, 커피 등의 정보이고 num은 동일 품목에서 더보기를 누를 때 나오는 페이지 정보다

#             # requests 정보 설정
#             # 현재 PC의 referer과 User-Agent 기입
#             headers = { 
#                         'referer': 'http://emart.ssg.com/best/main.ssg?Egnb=best',
#                         'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.54 Safari/537.36'
#                       }

#             seed = np.random.randint(100)
#             np.random.seed(seed)    
#             a = np.random.randint(5)
#             time.sleep(a)
#             try:
#                 response=requests.get(url, headers=headers)
#             except:
#                 print('접속 실패')
#                 break

#             # html 전환
#             data = BeautifulSoup(response.text, 'html.parser')

#             # 페이지별 이름, 가격, 평점 리스트 생성
#             product = [name.text for name in data.select('em.tx_ko')]
#             replace_name(product)   # 윗줄에서 정의한 제품명을 함수를 사용해 중복 데이터를 날려준다
            
#             price = [coin.text.strip() for coin in data.select('div.cunit_price.notranslate')]
#             remove_number(price)   # 위에서 정의한 함수를 사용해서 가격정보를 다듬는다.(할인으로 인한 중복 가격표기도 제거)
            
#             rate = [star.text.strip().strip().replace('별점 ','').replace('점','') for star in data.select('div.rate_bg')]
#             # 별점을 시각화나 데이터 작업에 용이하게 숫자만 남겨둔다
            
#             # 페이지 정보가 없으면 현재 검색중인 카테고리 루프문 종료
#             if not product:
#                 break;

#             else:
#                 # 현재 페이지 데이터프레임 생성(1개의 카테고리)
#                 df = pd.DataFrame(columns=['product','price','rate'], data=list(zip(product,price,rate)))

#                 # price, 별점 정보를 숫자형으로 전환
#                 df['price'] = df['price'].apply(remove_str).astype(int)
#                 df['rate'] = df['rate'].astype(float)

#                 # 현재 페이지 데이터프레임를 현재 카테고리 데이터프레임에 업데이트
#                 temp = df
#                 total_df = pd.concat([total_df, temp])
#                 total_df['카테고리'] = f'{best_list[best_list_link.index(link)]}'
                
#     # 카테고리별 작업 완료
#     print(f'{best_list[best_list_link.index(link)]} 인기품목 크롤링 완료')

#     # 전체 데이터프레임에 완료된 카테고리 데이터 업데이트
#     total_temp = total_df
#     best_total_df = pd.concat([best_total_df, total_temp])
#     best_total_df.index = range(len(best_total_df.index)) # 인덱스 초기화
#     best_total_df = best_total_df[['카테고리','product','price','rate']]
#     print(f'전체 데이터에 {best_list[best_list_link.index(link)]} 카테고리 병합 완료')
    
#     if link != '1000020119':
#         print('다음 카테고리 크롤링 진행\n')
#         time.sleep(17)         # 페이지 막힘 방지를 위해 17초 뒤에 다음 카테고리 실행
#     else:
#         print('***** 크롤링 작업 완료 *****\n')

# # 전체 카테고리 작업분 저장
# best_total_df.to_csv('/Users/ppangppang/Desktop/ssac/data_crwaling/ssg_test/databackup/today_test.csv',index=False)
# print('***** 데이터 저장 완료 *****')

## 유의사항
- 위에서 언급한 것처럼, 특정 브랜드가 새로 생길경우, 데이터가 밀려서 입력될 경우, 사전작업 탭의 3번 항목을 업데이트해주자
- 서버 접속때문인지, 한개의 카테고리가 그 양이 많을 경우, 서버가 막혀서 저장이 안될 경우가 있다.
- 위의 코드는 카테고리별로 25초의 간격을 두었으나, 데이터가 많은 등산용품의 마지막 페이지가 잘리는 경우가 생긴다. -> 데이터 중간중간 time.sleep을 걸어주는 것도 방법이겠다

#(부록) 코드 정보 및 코드 테스트
- 아래는 함수 지정 및 정상 작동 여부를 위해 따로 남겨둔 공간이다.
- 함수나 리스트 지정이 복잡해서 몇가지 예시를 남겨둔다.

In [ ]:
url=' http://emart.ssg.com/best/ajaxGetMoreItemList.ssg?cornrId=1000016472&cornrSetId=1000020109&dispCtgId=&pageSize=12&page=3'


# requests 정보 설정
headers = {
            'referer': 'http://emart.ssg.com/best/main.ssg?Egnb=best',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.54 Safari/537.36'
}

response=requests.get(url, headers=headers)

# html 전환
data = BeautifulSoup(response.text, 'html.parser')

# 페이지별 이름, 가격, 평점 리스트 생성
product = [name.text for name in data.select('em.tx_ko')]
price = [coin.text for coin in data.select('em.ssg_price')] 

In [ ]:
# em.ssg_price 이걸로 가격을 가져올경우, 할인가격과 기본가격이 따로 나눠져서 작업이 어렵다
price3 = [coin.text.strip() for coin in data.select('div.cunit_price.notranslate')]
price3

In [ ]:
ff=data.select('div.cunit_price.notranslate')
a=20
price3 = [coin.text.strip() for coin in data.select('div.cunit_price.notranslate')]
for i in range(len(price3)):
    if '에누리' in price3[i]:
        print(i)
        price3[i] = price3[i][0:25]
    price3[i] = price3[i][6:14].replace('\n','').replace('원','').replace('(','').replace('\\','').replace('￦','')
    
price3

In [ ]:
def remove_number(x):
    for i in range(len(x)):
        if '에누리' in x[i]:
            x[i] = x[i][0:25]
        x[i] = x[i][6:14].replace('\n','').replace('원','').replace('(','')
    return x

In [ ]:
ab=[1,2,3,4,5]
test=[1,2]
del ab[test[1]]

In [ ]:
time.sleep(10)
print(ab)

In [ ]:
# 링크 및 인덱스 호출 테스트
a=['a','b','c','d']
b=[1,2,3,4]

for i in a:
    print(f'tt{b[a.index(i)]}')
    
a.index('b')